<span style="font-size: 1.5em;">This code imports results from Predict in YOLO using a pickle file, selects the prediction per Defect_ID with the highest confidence, finds and saves corresponding cropped images of the defects and the full frame of the video at that point, and appends everything in a PDF document. The PDF requires formatting work. </span>

In [4]:
# Absolute paths to the Github repository
# Add yours and comment others

%cd C:\Users\s2018293031\Toon\Localenv\Gitrepo\Fenders_detection_Sohar
#%cd /Users/kroman/Documents/SOHAR/Localenvpc11

C:\Users\s2018293031\Toon\Localenv\Gitrepo\Fenders_detection_Sohar


In [5]:
from ultralytics import YOLO
import os

# Load a pretrained YOLOv8n-cls Classify model
# model_path = "C:/Users/s2018293031/Toon/Localenv/Localyolo/ultralytics-main/runs/train_old/best.pt"
# modelbest = YOLO(model_path)
# not necessary if in Git repository directory- uncomment the two lines above and comment out two lines below 

model_path = "Localyolo/ultralytics-main/runs/train_old/best.pt"
modelbest = YOLO(model_path)

# Run inference on a video
# video_path = "C:/Users/s2018293031/Toon/Localenv/Localyolo/TestVideos/Blanco/Side2.mp4"
# result_path = "C:/Users/s2018293031/Toon/Localenv/Localyolo/TestVideos/Predicted"

video_path = "Localyolo/TestVideos/Blanco/Side2.mp4"

result_path = "Localyolo/TestVideos/Predicted"

# Check if the directory exists
if not os.path.exists(result_path):
    # Create the directory if it doesn't exist
    os.makedirs(result_path)
    print(f"Directory '{result_path}' created.")
else:
    print(f"Directory '{result_path}' already exists.")


C:\Users\s2018293031\Toon\Localenv\local03\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Directory 'Localyolo/TestVideos/Predicted' already exists.


In [7]:
import pickle

# Load the Results object from the file
with open('results.pkl', 'rb') as f:
    results = pickle.load(f)

# Now you can use the 'results' variable for further analysis or processing
#print(results)

In [10]:
from datetime import timedelta
import cv2
import pandas as pd
from config import *

#this can be used later if we want to centralize video_paths and best_model from dictionaries stored in config.py
# def process_results(video_key, results):
#     video_path = video_paths.get(video_key)
#     if video_path is None:
#         raise ValueError(f"Video key '{video_key}' not found in config.py")

def process_results(video_path, results):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

    detection_data = []

    for frame_idx, result in enumerate(results):
        frame_time = frame_idx / fps

        boxes = result.boxes.xyxy

        if result.boxes.conf.numel() > 0:
            confidences = result.boxes.conf.tolist()
        else:
            confidences = None

        if result.boxes.cls.numel() > 0:
            classes = result.boxes.cls.tolist()
        else:
            classes = None

        if confidences is not None and classes is not None and boxes is not None and len(boxes) > 0:
            for box, conf, class_id in zip(boxes, confidences, classes):
                x_min, y_min, x_max, y_max = map(float, box[:4])
                class_name = modelbest.names[int(class_id)]

                detection_data.append({
                    "Timestamp": frame_time,
                    "Class": class_name,
                    "Confidence": conf,
                    "Bounding Box": (x_min, y_min, x_max, y_max)
                })

    df = pd.DataFrame(detection_data)

    df[["Timestamp", "Confidence"]] = df[["Timestamp", "Confidence"]].round(3)

    df['Frame Number'] = (df['Timestamp'] * fps + 1).astype(int)

    df.insert(0, "Fender ID", 1)
    df.insert(1, "Defect ID", 1)

    defect_id = 1
    last_timestamp = df.iloc[0]["Timestamp"]
    last_class = df.iloc[0]["Class"]

    for index, row in df.iterrows():
        time_difference = row["Timestamp"] - last_timestamp
        same_class = row["Class"] == last_class

        if time_difference > 0.15 or not same_class:
            defect_id += 1
        last_timestamp = row["Timestamp"]
        last_class = row["Class"]

        df.at[index, "Defect ID"] = defect_id

    cap.release()

    return df


{'video1': 'Localyolo/TestVideos/Blanco/Side2.mp4'}


In [11]:
df = process_results(video_path, results)  # Replace 'video_path' with the appropriate path

In [12]:
df.head(10)

,Fender ID,Defect ID,Timestamp,Class,Confidence,Bounding Box,Frame Number
0,1,1,0.000,Missing plate,0.331,"(970.7105712890625, 153.23175048828125, 1009.8...",1
1,1,1,0.033,Missing plate,0.308,"(971.4788818359375, 155.96726989746094, 1008.9...",1
2,1,1,0.067,Missing plate,0.305,"(970.7352905273438, 156.47772216796875, 1009.9...",3
3,1,1,0.100,Missing plate,0.330,"(971.4171752929688, 160.75662231445312, 1010.1...",4
4,1,1,0.133,Missing plate,0.323,"(972.0751953125, 163.16395568847656, 1011.5358...",4
5,1,1,0.167,Missing plate,0.252,"(971.1968994140625, 166.06741333007812, 1012.2...",6
6,1,1,0.200,Missing plate,0.292,"(971.5178833007812, 166.8504180908203, 1014.08...",7
7,1,1,0.233,Missing plate,0.328,"(971.8741455078125, 167.94607543945312, 1016.1...",7
8,1,1,0.267,Missing plate,0.348,"(972.6146850585938, 171.17660522460938, 1016.1...",9
9,1,1,0.300,Missing plate,0.368,"(973.2247924804688, 171.81591796875, 1016.0297...",10


In [13]:
top1_per_detection = df.groupby('Defect ID').apply(lambda group: group.nlargest(1, 'Confidence')).reset_index(drop=True)

#Directly apply the rounding
top1_per_detection['Bounding Box'] = top1_per_detection['Bounding Box'].apply(lambda x: tuple(map(lambda y: round(y, 3), x)))


In [14]:
top1_per_detection.head(7)

,Fender ID,Defect ID,Timestamp,Class,Confidence,Bounding Box,Frame Number
0,1,1,3.233,Missing plate,0.599,"(1091.771, 598.786, 1187.942, 874.432)",97
1,1,2,6.833,Missing plate,0.270,"(1073.213, 337.165, 1115.99, 483.438)",205
2,1,3,7.000,Loose plate,0.255,"(1047.939, 289.87, 1094.46, 399.043)",211
3,1,4,7.467,Missing plate,0.264,"(1102.574, 442.43, 1150.099, 613.725)",225
4,1,5,7.667,Missing plate,0.254,"(1114.5, 485.426, 1167.469, 667.938)",231
5,1,6,7.967,Missing plate,0.263,"(1130.006, 552.129, 1194.09, 760.207)",240
6,1,7,8.833,Missing plate,0.386,"(1225.257, 869.698, 1321.212, 1072.354)",265


In [15]:
print(results[0].save_dir)

Localyolo\TestVideos\Predicted\predict


In [16]:
import os
import shutil

# Set the directory where the cropped images are stored
crops_directory = os.path.join(results[0].save_dir, "crops")
#print(crops_directory)

# Extracting the video file name from video_path
video_file_name = os.path.splitext(os.path.basename(video_path))[0]
#print(video_file_name)

# Create a new directory to store filtered crops
filtered_crops_directory = os.path.join(results[0].save_dir, "crops_filtered")
os.makedirs(filtered_crops_directory, exist_ok=True)

for index, row in top1_per_detection.iterrows():
    frame_number = row["Frame Number"]
    class_name = row["Class"]

    # Construct the file path for the cropped image in the original crops directory
    original_image_path = os.path.join(crops_directory, class_name, f"{video_file_name}_{frame_number}.jpg")
    print(original_image_path)
    # Construct the file path for the cropped image in the new filtered crops directory
    filtered_image_path = os.path.join(filtered_crops_directory, f"{video_file_name}_{frame_number}.jpg")
    #print(filtered_image_path)  # Optional: Print the path for verification

    # Check if the file exists before copying
    if os.path.exists(original_image_path):
        # Copy the file to the filtered crops directory
        shutil.copy(original_image_path, filtered_image_path)
    #sometimes there are some problems with indexing, so I also check for the next few crops, and save the first available one
    else:
    # Try frames between -2 and +2
        for i in range(-2, 3):
            next_frame_number = frame_number + i
            next_image_path = os.path.join(crops_directory, class_name, f"{video_file_name}_{next_frame_number}.jpg")
            if os.path.exists(next_image_path):
                # Copy the file to the filtered crops directory
                shutil.copy(next_image_path, filtered_image_path)
                break  # Stop trying once a valid file is found


Localyolo\TestVideos\Predicted\predict\crops\Missing plate\Side2_97.jpg
Localyolo\TestVideos\Predicted\predict\crops\Missing plate\Side2_205.jpg
Localyolo\TestVideos\Predicted\predict\crops\Loose plate\Side2_211.jpg
Localyolo\TestVideos\Predicted\predict\crops\Missing plate\Side2_225.jpg
Localyolo\TestVideos\Predicted\predict\crops\Missing plate\Side2_231.jpg
Localyolo\TestVideos\Predicted\predict\crops\Missing plate\Side2_240.jpg
Localyolo\TestVideos\Predicted\predict\crops\Missing plate\Side2_265.jpg


<span style="font-size: 1.5em;">The method above probably requires some checking for robustness, but it works for now.</span>

In [17]:
#Code below saves frames of the full video when detections are recorded

import os
import cv2
import pandas as pd

#folder for full saved frames
full_frames_folder_name = "detection_full_frames"
video_path_boxes = f"{results[0].save_dir}/Side2.mp4"

# Create a new directory to store full frames corresponding to detections
full_frames_directory = os.path.join(results[0].save_dir, full_frames_folder_name)
os.makedirs(full_frames_directory, exist_ok=True)

def capture_frames(video_path, frame_numbers, save_dir):
    cap = cv2.VideoCapture(video_path_boxes)
    fps = cap.get(cv2.CAP_PROP_FPS)

    for frame_number in frame_numbers:
        # Set the frame number to capture
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

        # Read the frame
        ret, frame = cap.read()

        if ret:
            # Construct the file path to save the frame
            frame_path = os.path.join(save_dir, f"frame_{frame_number}.jpg")

            # Save the frame
            cv2.imwrite(frame_path, frame)
            print(f"Saved frame {frame_number} to {frame_path}")

    cap.release()

# Assuming `top1_per_detection` is already defined
frame_numbers_to_capture = top1_per_detection["Frame Number"].tolist()

# Capture frames and save them in the "detection_full_frames" directory
capture_frames(video_path, frame_numbers_to_capture, full_frames_directory)

In [18]:
print(top1_per_detection.columns)

Index(['Fender ID', 'Defect ID', 'Timestamp', 'Class', 'Confidence',
       'Bounding Box', 'Frame Number'],
      dtype='object')


In [163]:
print(results[0].save_dir)

Localyolo/TestVideos/Predicted/predict10


In [19]:
import os
import cv2
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from PyPDF2 import PdfMerger
import pandas as pd

# Set up PDF file
path_topdf = results[0].save_dir
pdf_filename = f"{path_topdf}/detection_report_with_images.pdf"

# Check if the PDF file already exists
if not os.path.exists(pdf_filename):
    # Create a new PDF file if it doesn't exist
    pdf_pages = PdfPages(pdf_filename)
else:
    # Open the existing PDF file in append mode
    pdf_pages = PdfPages(pdf_filename, 'a')

# Create a PdfMerger object
merger = PdfMerger()

# Iterate through each row of top1_per_detection
for index, row in top1_per_detection.iterrows():
    # Create a figure and axes for the table
    fig, ax = plt.subplots(figsize=(8.5, 11))
    ax.axis('off')  # Turn off axis labels and ticks

    # Create a table from the current row of top1_per_detection and add it to the axes
    table_data = top1_per_detection.loc[[index]].values
    table_columns = top1_per_detection.columns
    table = ax.table(cellText=table_data, colLabels=table_columns, cellLoc='center', loc='center')

    # Style the table
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)  # Increase table size for better readability

    # Adjust column widths based on content
    table.auto_set_column_width([i for i in range(len(table_columns))])

    # Save the current figure to the PDF file
    pdf_pages.savefig(fig, bbox_inches='tight')
    plt.close()

    # Add the cropped image from crops_filtered folder
    cropped_image_path = os.path.join(results[0].save_dir, 'crops_filtered', f"{video_file_name}_{row['Frame Number']}.jpg")
    if os.path.exists(cropped_image_path):
        img = cv2.imread(cropped_image_path)
        plt.figure(figsize=(8.5, 11))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title("Cropped Image")
        pdf_pages.savefig(bbox_inches='tight')
        plt.close()

    # Add the full image from detection_full_frames folder
    full_image_path = os.path.join(results[0].save_dir, 'detection_full_frames', f"frame_{row['Frame Number']}.jpg")
    if os.path.exists(full_image_path):
        img = cv2.imread(full_image_path)
        plt.figure(figsize=(8.5, 11))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title("Full Image")
        pdf_pages.savefig(bbox_inches='tight')
        plt.close()

# Close the PDF file
pdf_pages.close()

# Optionally, you can add a title page to the PDF using reportlab
title_page_filename = f"{path_topdf}/title_page.pdf"
canvas_obj = canvas.Canvas(title_page_filename, pagesize=letter)
canvas_obj.setFont("Helvetica", 14)
canvas_obj.drawCentredString(300, 700, "Detection Report")
canvas_obj.save()

# Create a PdfMerger object
merger = PdfMerger()

# Add the title page and dataframe to the merged PDF
merger.append(title_page_filename)
merger.append(pdf_filename)

# Output the final merged PDF
merged_pdf_filename = f"{path_topdf}/final_report.pdf"
merger.write(merged_pdf_filename)
merger.close()


***Below, I started definying unit tests to tests that the post-processing code is doing what we expect it to do:
However, it is not working as expected yet, so ignore these lines for now***

In [ ]:
!jupyter nbconvert --to script post_processing.ipynb

In [ ]:
import unittest
from test_detection import *

# Print a message indicating whether the tests passed or failed
try:
    from test_detection import TestFrameNumber
    suite = unittest.TestLoader().loadTestsFromTestCase(TestFrameNumber)
    result = unittest.TextTestRunner(verbosity=4).run(suite)

    if not result.errors and not result.failures:
        print("All unit tests passed!")
    else:
        print("Some unit tests failed.")
        for test_case, traceback in result.failures + result.errors:
            test_name = str(test_case).split(' ')[0]
            print(f"Test {test_name} failed.")
except ImportError:
    print("Unit tests were not executed.")
